# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from citipy import citipy
#import os  #why do I need this?
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("Results/Cities.csv")
cities_df

,City Number,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1.0,Hobart,90.0,1586884824,82.0,-42.88,147.33,15.56,3.60
1,2.0,Punta Arenas,75.0,1586884830,62.0,-53.15,-70.92,12.00,8.70
2,3.0,Kurumkan,88.0,1586884831,54.0,54.30,110.30,3.01,1.78
3,4.0,Temerloh,86.0,1586884831,88.0,3.45,102.42,24.01,0.52
4,5.0,Castro,89.0,1586884831,53.0,-24.79,-50.01,20.48,3.23
...,...,...,...,...,...,...,...,...,...
573,574.0,Contamana,27.0,1586884939,37.0,-7.33,-75.02,34.20,3.49
574,575.0,Torremolinos,20.0,1586884851,77.0,36.62,-4.50,18.00,3.60
575,576.0,Ahipara,68.0,1586884939,72.0,-35.17,173.17,15.33,7.39
576,577.0,Necochea,0.0,1586884939,66.0,-38.55,-58.74,19.44,4.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[["Lat", "Lng"]]
humidity= cities_df["Humidity"]

#Display map
fig=gmaps.figure()

#Alternative to make a smaller figure
# figure_layout={
#     "width": "400px",
#     "height": "300px",
#     "border": "1px solid black",
#     "padding": "1px",
#     "margin": "0 auto 0 auto"}
# fig= gmaps.figure(layout= figure_layout)



# Create heat layer
heat_layer=gmaps.heatmap_layer(locations=locations, weights=humidity, \
                               dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
best_cities_df=cities_df.loc[(cities_df["Max Temp"] <=25) & (cities_df["Max Temp"] >=20) &\
                         (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] <10)]
best_cities_df= best_cities_df.reset_index(drop=True)
best_cities_df

,City Number,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,29.0,Gaza,0.0,1586884836,72.0,31.50,34.47,20.00,3.60
1,41.0,Nouadhibou,0.0,1586884838,49.0,20.93,-17.03,23.00,9.80
2,47.0,Nosy Varika,0.0,1586884840,84.0,-20.58,48.53,24.16,0.34
3,159.0,Tikrit,0.0,1586884862,26.0,34.61,43.68,21.47,3.49
4,219.0,Behbahān,0.0,1586884874,22.0,30.60,50.24,25.00,4.10
5,287.0,Saldanha,0.0,1586884696,60.0,-33.01,17.94,20.00,6.70
6,324.0,Presidencia Roque Sáenz Peña,0.0,1586884895,27.0,-26.79,-60.44,24.41,3.52
7,441.0,Umluj,0.0,1586884916,49.0,25.02,37.27,21.71,2.78
8,484.0,Kiryat Gat,0.0,1586884923,72.0,31.61,34.76,20.00,3.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df= pd.DataFrame({
    "Hotel Name":[],
    "City":[],
    "Country":[],
    "Lat":[],
    "Lng":[]
})
hotel_df

,Hotel Name,City,Country,Lat,Lng


In [6]:
query_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params={
    "location":"",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
}


for index, row in best_cities_df.iterrows():
    name= row["City"]
    coordinates=(f'{row["Lat"]}, {row["Lng"]}')
    params["location"]= coordinates
    print(f'Retrieving results for index {index}: {name}')
    response=requests.get(query_url, params=params).json()
    results=response["results"]
    
    try:
        print(f'The first hotel nearby {name} is {results[0]["name"]}')
        hotel_df.loc[index, "Hotel Name"]= results[0]["name"]
        hotel_df.loc[index, "Lat"]= row["Lat"]
        hotel_df.loc[index, "Lng"]= row["Lng"]
        city=citipy.nearest_city(row["Lat"], row["Lng"])
        hotel_df.loc[index, "City"]=city.city_name.capitalize()
        hotel_df.loc[index, "Country"]= city.country_code.upper()
        
    except (KeyError, IndexError):
        print(f'----Missing field/result for {name}. Skipping----')

hotel_df
# pprint(results[0])
# city=citipy.nearest_city(row["Lat"], row["Lng"])
# city.country_code.upper()

Retrieving results for index 0: Gaza
The first hotel nearby Gaza is فندق جراند بالاس Grand Palace Hotel
Retrieving results for index 1: Nouadhibou
The first hotel nearby Nouadhibou is Free Zone Hotel Nouadhibou
Retrieving results for index 2: Nosy Varika
The first hotel nearby Nosy Varika is Hotel Volazara
Retrieving results for index 3: Tikrit
The first hotel nearby Tikrit is فندق تكريت بلازا
Retrieving results for index 4: Behbahān
The first hotel nearby Behbahān is هتل و رستوران بزرگ مارون
Retrieving results for index 5: Saldanha
The first hotel nearby Saldanha is Whispering Whale Luxury Apartments
Retrieving results for index 6: Presidencia Roque Sáenz Peña
The first hotel nearby Presidencia Roque Sáenz Peña is Hotel Gualok
Retrieving results for index 7: Umluj
The first hotel nearby Umluj is عروس الشاطئ للوحدات السكنية المفروشة
Retrieving results for index 8: Kiryat Gat
The first hotel nearby Kiryat Gat is Desert Gat


,Hotel Name,City,Country,Lat,Lng
0,فندق جراند بالاس Grand Palace Hotel,Sederot,IL,31.50,34.47
1,Free Zone Hotel Nouadhibou,Nouadhibou,MR,20.93,-17.03
2,Hotel Volazara,Nosy varika,MG,-20.58,48.53
3,فندق تكريت بلازا,Tikrit,IQ,34.61,43.68
4,هتل و رستوران بزرگ مارون,Behbahan,IR,30.60,50.24
5,Whispering Whale Luxury Apartments,Saldanha,ZA,-33.01,17.94
6,Hotel Gualok,Presidencia roque saenz pena,AR,-26.79,-60.44
7,عروس الشاطئ للوحدات السكنية المفروشة,Umm lajj,SA,25.02,37.27
8,Desert Gat,Qiryat gat,IL,31.61,34.76


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))